In [1]:
import os
import keras
import numpy as np
np.random.seed(135109)

Using TensorFlow backend.


In [2]:
sensor = '02'

In [3]:
def dataset():
    X = []
    y = []
    data_dir_0 = sorted(os.listdir(os.path.join('pac_data', sensor, '0')))   
    i = 0
    for f in data_dir_0:
        if i % 100 == 0: print(i)
        i += 1
        X.append(np.load(os.path.join('pac_data', sensor, '0', f))['x'])
        y.append(0)
    X = np.asarray(X, dtype=np.float16)
    y = np.asarray(y, dtype=int)
    indices = np.arange(y.shape[0])
    np.random.shuffle(indices)
    X_train = X[indices[:int(0.8*len(indices))]]
    y_train = y[indices[:int(0.8*len(indices))]]
    X_val = X[indices[int(0.8*len(indices)):]]
    y_val = y[indices[int(0.8*len(indices)):]]
    labels_present = os.listdir(os.path.join('pac_data', sensor))
    if '1' in labels_present:
        X = []
        y = []
        data_dir_1 = sorted(os.listdir(os.path.join('pac_data', sensor, '1')))
        i = 0
        for f in data_dir_1:
            if i % 100 == 0: print(i)
            i += 1
            X.append(np.load(os.path.join('pac_data', sensor, '1', f))['x'])
            y.append(1)        
        X = np.asarray(X, dtype=np.float16)
        y = np.asarray(y, dtype=int)
        indices = np.arange(y.shape[0])
        np.random.shuffle(indices)
        X_train = np.concatenate([X_train, X[indices[:int(0.8*len(indices))]]])
        y_train = np.concatenate([y_train, y[indices[:int(0.8*len(indices))]]])
        X_val = np.concatenate([X_val, X[indices[int(0.8*len(indices)):]]])
        y_val = np.concatenate([y_val, y[indices[int(0.8*len(indices)):]]])
    X_train = np.expand_dims(np.asarray(X_train, dtype=np.float16), axis=3)
    y_train = np.asarray(y_train, dtype=int)
    X_val = np.expand_dims(np.asarray(X_val, dtype=np.float16), axis=3)
    y_val = np.asarray(y_val, dtype=int)
    X_train /= 5.
    X_val /= 5.
    mean_arr = np.mean(X_train, axis = 0)
    np.save(os.path.join('pac_data', sensor, 'mean.npy'), mean_arr)
    X_train -= mean_arr 
    X_val -= mean_arr  
    return X_train, y_train, X_val, y_val

In [4]:
X_train, y_train, X_val, y_val = dataset()

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
0
100
200
300
400
500
600
700
800
900


In [5]:
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(9080, 240, 320, 1) (9080,) (2271, 240, 320, 1) (2271,)


In [6]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())

['/gpu:0']


In [7]:

from keras.models import Sequential
from keras.layers import TimeDistributed, GlobalAveragePooling2D, Activation, Dense, Input
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers.recurrent import LSTM
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.layers import Dropout, Flatten
from keras.layers import concatenate
from keras import regularizers
from keras import initializers
from keras import constraints
from keras.models import Model
from keras import backend as K
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras.utils import conv_utils
from keras.engine.topology import get_source_inputs
from keras.engine import InputSpec
from keras.applications.xception import Xception
from keras import metrics

In [8]:
#linear
def linear(dropout):
    inputs = Input(shape=(240, 320, 1))
    x = Flatten()(inputs)
    x = Dropout(dropout)(x)
    x = Dense(2, activation='softmax')(x)
    model = Model(inputs, x, name='linear')
    return model

In [9]:
from keras import layers
from keras.layers import Activation
from keras.layers import SeparableConv2D

def xception(dropout):
    inputs = Input(shape=(240, 320, 1))    

    x = Conv2D(32, (3, 3), padding='same', strides=(2, 2), use_bias=False, name='block1_conv1')(inputs)
    x = BatchNormalization(name='block1_conv1_bn')(x)
    x = Activation('relu', name='block1_conv1_act')(x)
    x = Conv2D(64, (3, 3), padding='same', use_bias=False, name='block1_conv2')(x)
    x = BatchNormalization(name='block1_conv2_bn')(x)
    x = Activation('relu', name='block1_conv2_act')(x)

    residual = Conv2D(128, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False, name='block2_sepconv1')(x)
    x = BatchNormalization(name='block2_sepconv1_bn')(x)
    x = Activation('relu', name='block2_sepconv2_act')(x)
    x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False, name='block2_sepconv2')(x)
    x = BatchNormalization(name='block2_sepconv2_bn')(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='block2_pool')(x)
    x = layers.add([x, residual])

    residual = Conv2D(256, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = Activation('relu', name='block3_sepconv1_act')(x)
    x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False, name='block3_sepconv1')(x)
    x = BatchNormalization(name='block3_sepconv1_bn')(x)
    x = Activation('relu', name='block3_sepconv2_act')(x)
    x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False, name='block3_sepconv2')(x)
    x = BatchNormalization(name='block3_sepconv2_bn')(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='block3_pool')(x)
    x = layers.add([x, residual])

    residual = Conv2D(728, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = Activation('relu', name='block4_sepconv1_act')(x)
    x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name='block4_sepconv1')(x)
    x = BatchNormalization(name='block4_sepconv1_bn')(x)
    x = Activation('relu', name='block4_sepconv2_act')(x)
    x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name='block4_sepconv2')(x)
    x = BatchNormalization(name='block4_sepconv2_bn')(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='block4_pool')(x)
    x = layers.add([x, residual])

    for i in range(8):
        residual = x
        prefix = 'block' + str(i + 5)

        x = Activation('relu', name=prefix + '_sepconv1_act')(x)
        x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name=prefix + '_sepconv1')(x)
        x = BatchNormalization(name=prefix + '_sepconv1_bn')(x)
        x = Activation('relu', name=prefix + '_sepconv2_act')(x)
        x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name=prefix + '_sepconv2')(x)
        x = BatchNormalization(name=prefix + '_sepconv2_bn')(x)
        x = Activation('relu', name=prefix + '_sepconv3_act')(x)
        x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name=prefix + '_sepconv3')(x)
        x = BatchNormalization(name=prefix + '_sepconv3_bn')(x)

        x = layers.add([x, residual])

    residual = Conv2D(1024, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = Activation('relu', name='block13_sepconv1_act')(x)
    x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name='block13_sepconv1')(x)
    x = BatchNormalization(name='block13_sepconv1_bn')(x)
    x = Activation('relu', name='block13_sepconv2_act')(x)
    x = SeparableConv2D(1024, (3, 3), padding='same', use_bias=False, name='block13_sepconv2')(x)
    x = BatchNormalization(name='block13_sepconv2_bn')(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='block13_pool')(x)
    x = layers.add([x, residual])

    x = SeparableConv2D(1536, (3, 3), padding='same', use_bias=False, name='block14_sepconv1')(x)
    x = BatchNormalization(name='block14_sepconv1_bn')(x)
    x = Activation('relu', name='block14_sepconv1_act')(x)

    x = SeparableConv2D(2048, (3, 3), padding='same', use_bias=False, name='block14_sepconv2')(x)
    x = BatchNormalization(name='block14_sepconv2_bn')(x)
    x = Activation('relu', name='block14_sepconv2_act')(x)

    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dropout(dropout)(x)
    x = Dense(2, activation='softmax', name='predictions')(x)
    model = Model(inputs, x, name='xception')
    return model


'''
def xception(dropout):
    base_model = Xception(include_top=False, input_shape = (240, 320, 1), pooling='avg')
    x = base_model.output
    x = Flatten()(x)
    x = Dropout(dropout)(x) # drop prob
    predictions = Dense(2, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions, name='linear')
    return model
'''

"\ndef xception(dropout):\n    base_model = Xception(include_top=False, input_shape = (240, 320, 1), pooling='avg')\n    x = base_model.output\n    x = Flatten()(x)\n    x = Dropout(dropout)(x) # drop prob\n    predictions = Dense(2, activation='softmax')(x)\n    model = Model(inputs=base_model.input, outputs=predictions, name='linear')\n    return model\n"

In [10]:
def vanillacnn(dropout):
    inputs = Input(shape=(240, 320, 1)) 
    x = Conv2D(32, (3, 3), strides=(2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)
    x = Dropout(dropout)(x)
    
    x = Conv2D(64, (3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)
    x = Dropout(dropout)(x)
    
    x = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)
    x = Dropout(dropout)(x)
    
    x = Conv2D(256, (3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)
    
    x = Flatten()(x)
    x = Dropout(dropout)(x)
    x = Dense(2, activation='softmax', name='predictions')(x)
    model = Model(inputs, x, name='xception')
    return model
    
    

In [11]:
model_name = 'model' + sensor + '.h5'
model = vanillacnn(0.5)

adam = keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=[metrics.sparse_categorical_accuracy])


checkpoint = ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
stop = EarlyStopping(patience=20, verbose=1)

callbacks_list = [checkpoint, stop]

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 240, 320, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 120, 160, 32)      320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 120, 160, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 120, 160, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 60, 80, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 80, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 40, 64)        18496     
__________

In [12]:
model.fit(x=X_train, 
          y=y_train, 
          batch_size=32, 
          epochs=1000, 
          verbose=1, 
          callbacks=callbacks_list, 
          validation_data=(X_val, y_val), 
          shuffle=True)


Train on 9080 samples, validate on 2271 samples
Epoch 1/1000
9080/9080 [==============================] - 30s - loss: 0.2828 - sparse_categorical_accuracy: 0.9126 - val_loss: 0.2873 - val_sparse_categorical_accuracy: 0.9128
Epoch 2/1000
9080/9080 [==============================] - 29s - loss: 0.1649 - sparse_categorical_accuracy: 0.9404 - val_loss: 0.1369 - val_sparse_categorical_accuracy: 0.9304
Epoch 3/1000
9080/9080 [==============================] - 29s - loss: 0.1070 - sparse_categorical_accuracy: 0.9574 - val_loss: 0.0732 - val_sparse_categorical_accuracy: 0.9731
Epoch 4/1000
9080/9080 [==============================] - 29s - loss: 0.0842 - sparse_categorical_accuracy: 0.9698 - val_loss: 0.0619 - val_sparse_categorical_accuracy: 0.9753
Epoch 5/1000
9080/9080 [==============================] - 29s - loss: 0.0705 - sparse_categorical_accuracy: 0.9751 - val_loss: 0.0603 - val_sparse_categorical_accuracy: 0.9797
Epoch 6/1000
9080/9080 [==============================] - 29s - loss: 0.

In [13]:
from keras.models import load_model
trained_model = load_model(model_name)
print(trained_model.metrics_names)
trained_model.evaluate(x=X_train, y=y_train)


['loss', 'sparse_categorical_accuracy']
9056/9080 [============================>.] - ETA: 0s

[0.013383198205294706, 0.99570484581497798]

In [14]:
trained_model.evaluate(x=X_val, y=y_val)

2271/2271 [==============================] - 3s     


[0.043335138333958599, 0.98590929027382679]